<a href="https://colab.research.google.com/github/leonidsanov/apache_spark/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа

Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

Данные для таблицы:

In [ ]:
import pandas as pd

In [ ]:
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary':['70', '80', '60', '90'], 'ManagerId':['3', '4', 'Null', 'Null']}
df = pd.DataFrame(data)
df

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


Результат должен быть:
Joe (табличка с одной строкой и одним столбцом со значением Joe)


In [ ]:
!pip install pyspark >> None

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType

In [ ]:
# Инициализация Spark Session
spark = SparkSession.builder.appName("employees").getOrCreate()

In [ ]:
# Создаём dataframe spark из dataframe pandas
spark_df = spark.createDataFrame(df)

In [ ]:
# Показать исходный DataFrame
spark_df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



In [ ]:
# Преобразование ManagerId в числовой формат и объединение таблицы с самой собой для сравнения зарплат
spark_df = spark_df.withColumn("ManagerId", col("ManagerId").cast("integer"))
spark_df.alias("emp").join(spark_df.alias("mgr"), col("emp.ManagerId") == col("mgr.id"), "left_outer") \
    .select(col("emp.Name").alias("Employee"), col("emp.Salary").alias("Emp_Salary"),
            col("mgr.Salary").alias("Mgr_Salary")) \
    .where(col("Emp_Salary") > col("Mgr_Salary")) \
    .select("Employee").show()

+--------+
|Employee|
+--------+
|     Joe|
+--------+

